# Solar model - version 2

Date of creation: 17.06.2020

Last updated: 4.07.2020

In [1]:
import math
import numpy as np
import pandas as pd

%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
h = 1.054E-27 # reduced Planck constant
c = 3e10 # speed of light
G = 6.67E-8 # gravitational constant
kB = 1.381E-16 # Boltzmann constant
m_prot = 1.67E-24 # масса протона
m_elec = 9.11E-28 # масса электрона
alpha = 137 # e2/h/c - постоянная тонкой структуры
e2 = h*c/alpha
r_elec = e2/m_elec/c/c # классический радиус электрона
pi = math.pi # 3.14...
sigmaT = 8*pi/3*r_elec*r_elec # сечение томсоновского рассеяния на электронах
kappaT = sigmaT/m_prot
sigma = pow(pi, 2)*pow(kB, 4)/60*pow(h, -3)*pow(c, -2) # sigma*T^4
a = pow(pi, 2)*pow(kB, 4)/15*pow(h, -3)*pow(c, -3)
gamma = 5/3

In [3]:
M_sol = 1.99e33 # solar mass
R_sol = 6.96e10 # solar radius
L_sol = 3.85e33 # solar luminosity
Teff = pow(L_sol/4/pi/sigma/R_sol/R_sol, 1/4) # 5780 solar surface temperature

## Функции для модели

In [46]:
# Эволюция содержания водорода
dimM = 200
dimT = 50
Xpre = 0.732
xmt = np.full((dimM, dimT), Xpre)
epoch = 0 # 0 - dimT-1 
year = 365.25*24*3600
eVolt = 1.6e-12 # ergs
step_t = 1e9*year # step by time
dEpp = (26.23 - 0.5)*1e6*eVolt

In [64]:
epoch = 0

In [6]:
# Интерполяция содержания водорода для данной массы
def XYZ(M, Z=0.02, dimM=200) :
    i = int(M*dimM)
    if i > dimM - 1 :
        i = dimM - 1
    X = xmt[i, epoch]
    return (X, 1-Z-X, Z)

In [7]:
# for i in range(100):
#     xmt[i, epoch] = 0.

In [65]:
xmt[:,epoch]

array([0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732,
       0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732,
       0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732,
       0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732,
       0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732,
       0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732,
       0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732,
       0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732,
       0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732,
       0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732,
       0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732,
       0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732,
       0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732,
       0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732,
       0.732, 0.732,

In [91]:
def MU(X, Y, Z) :
    return  1/(2*X + 3/4*Y + 1/2*Z)

# Табличные данные для функции непрозрачности
Tst = [2120000.,  2720000.,  3400000.,  4640000.,  6030000.,
    7270000.,  8000000.,  8770000.,  9600000., 10200000., 10800000.,
   11400000., 12400000., 13800000., 14800000., 15500000.]
Kst = [9.94082840e+01, 3.60946746e+01,
   1.03492885e+01, 1.64912281e+00, 4.35865504e-01, 1.73913043e-01,
   1.08597285e-01, 7.14285714e-02, 4.93506494e-02, 3.93873085e-02,
   3.17164179e-02, 2.53164557e-02, 1.77439797e-02, 1.20259019e-02,
   8.96191187e-03, 7.03774792e-03]
def kappa(den, T, X, Y, Z) :
#     den *= 0.1
    if T <= Tst[0] :
#         return den*Kst[0]*T/Tst[0]
        return den*Kst[0]
    for i in range(len(Tst) - 1) :
        if T <= Tst[i+1] :
            return den*(Kst[i] + (T - Tst[i])/(Tst[i+1] - Tst[i])*(Kst[i+1] - Kst[i]))
    return den*Kst[-1]

In [10]:
# Уравнение состояния 
def Pressure(den, T, X, Y, Z) :
    mu = 1/(2*X + 3/4*Y + 1/2*Z)
    return den/m_prot/mu*kB*T

In [85]:
# Скорость энерговыделения в p-p реакции [эрг/с/г] деленная на X
def E0pp(den, T, X) :
    T0 = (1e6, 5e6, 10e6, 15e6, 20e6, 30e6)
    e0 = (4e-9, 1.8e-3, 6.8e-2, 0.377, 1.09, 4.01)
    n = (10.6, 5.95, 4.60, 3.95, 3.64, 3.03)
    found = False
    for i in range (len(T0) - 1) :
        if T < pow(T0[i]*T0[i+1], 0.5) :
            found = True
            break;
        if not found :
            i = len(T0) - 1
    # print('i=%d T0=%.1e e0=%.3e n=%.2f' % (i, T0[i], e0[i], n[i]))
    return den*X*e0[i]*pow(T/T0[i], n[i])

# Скорость энерговыделения в CNO цикле [эрг/с/г] деленная на X
def E0cno(den, T, XCNO) :
    T0 = (6e6, 10e6, 15e6, 20e6, 30e6, 50e6, 100e6)
    e0 = (9e-10, 3.4e-4, 1.94, 4.5e2, 4.1e5, 6.2e8, 1.9e12)
    n = (27.3, 22.9, 19.9, 18.0, 15.6, 13.6, 10.2)
    found = False
    for i in range (len(T0) - 1) :
        if T < pow(T0[i]*T0[i+1], 0.5) :
            found = True
            break;
    if not found :
        i = len(T0) - 1 
    # print('i=%d T0=%.1e e0=%.3e n=%.2f' % (i, T0[i], e0[i], n[i]))
    return den*XCNO*e0[i]*pow(T/T0[i], n[i])

# For best fit to SSM
def Etot(den, T, X, Y, Z) :
    return X*E0pp(den, T, X) + X*E0cno(den, T, Z)
#     return 1.3*Epp(den, T, X) + 3.5*Ecno(den, T, X, Z)

## Интегрирование от центра по расстоянию
параметры модели: плотность и температура в центре.

хим.состав, число точек, шаг по радиусу.

In [60]:
def model_R(den0, T0, Z=0.02, dim=30000, dimM=200, step=R_sol/10000, debug=False) :
    dr = step  
    # distance from the center
    r = np.zeros(dim, dtype='float64')
    # density
    d = np.zeros(dim, dtype='float64')
    # temperature
    t = np.zeros(dim, dtype='float64')
    # светимость
    l = np.zeros(dim, dtype='float64')
    # mass inside r
    m = np.zeros(dim, dtype='float64')
    # pressure
    p = np.zeros(dim, dtype='float64')
    # hydrogen
    x = np.zeros(dim, dtype='float64')
    # opacity
    k = np.zeros(dim, dtype='float64')
    # 1 - convection, 0 - radiation
    conv = np.zeros(dim)
    # скорость выгорания водорода (еличина, обратная характерному времени)
    v = np.zeros(dim, dtype='float64')
    
# начальные данные
    
    # Step 1:
    for n in range(2) :
        r[n] = 10*n*dr
        d[n] = den0
        t[n] = T0
        V = 4*pi/3*pow(r[n], 3)
        m[n] = V*d[n]
        X, Y, Z = XYZ(m[n]/M_sol) # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        x[n] = X
        mu = MU(X, Y, Z)
        l[n] = m[n]*Etot(d[n], t[n], X, Y, Z)
        p[n] = Pressure(d[n], t[n], X, Y, Z)
        k[n] = kappa(d[n], t[n], X, Y, Z)
        v[n] = 4*m_prot*(E0pp(d[n], t[n], x[n]) + E0cno(d[n], t[n], 0.02))*year/dEpp
# Ckeck for convection:
        Tgrad = k[n]*d[n]*l[n]/(4*a*c*pow(t[n], 3))
        cond = 8*pi/15*G*m_prot*mu/kB
        if debug :
            print(X, Y, Z, mu)
            print("Center: Tgrad=%.4e cond=%.4e" % (Tgrad, cond))
        if Tgrad > cond :
            conv[n] = 1
        else :
            conv[n] = 0
        if debug :
            print('Center: pressure=%e opacity=%.3f X=%.3f mu=%.5f' % (p[n], k[n], X, mu))
            if conv[n]:
                print("Convection")
            else:
                print("Radiation")
    warn = True
    for n in range(1, dim - 1) :
        X, Y, Z = XYZ(m[n]/M_sol)
        mu = MU(X, Y, Z)
        x[n] = X
        r[n+1] = r[n] + dr 
        S = 4*pi*r[n]*r[n]
        g = G*m[n]/r[n]/r[n]
        dp = -g*d[n]*dr
        p[n+1] = p[n] + dp
        k[n] = kappa(d[n], t[n], X, Y, Z)
        if debug :
            print('n=%d r/R_sol=%.4f m/M_sol=%.4f k=%.3f den=%.2f T=%.3e' % (n, r[n]/R_sol, m[n]/M_sol, k[n], d[n], t[n]))
        dt = -3/16*k[n]*d[n]*l[n]/sigma/pow(t[n], 3)/S*dr
        if dt < -2/5*g*m_prot*mu/kB*dr : # Условие конвекции
            dTconv = 2/5*t[n]/p[n]*dp
            if conv[n] == 0:
                if debug :
                    print("Begin convection zone at R=%.3f" % (r[n]/R_sol))
                    print("dT=%e porog=%e n=%d dtConv=%e" % (dt, -2/5*g*m_prot*mu/kB*dr, n, dTconv))
            conv[n+1] = 1
            dt = dTconv
        else :
            if conv[n] :
                if debug:
                    print("End convection zone at R=%.3f" % (r[n]/R_sol))
            conv[n+1] = 0
        
        t[n+1] = t[n] + dt
        # Находим плотность из уравнения состояния
        d[n+1] = mu*m_prot*p[n+1]/kB/t[n+1]
        davg = 0.5*(d[n] + d[n+1])
        tavg = 0.5*(t[n] + t[n+1])
        m[n+1] = m[n] + S*dr*davg
        l[n+1] = l[n] + S*dr*davg*Etot(davg, tavg, X, Y, Z)
        v[n] = 4*m_prot*(E0pp(d[n], t[n], x[n]) + E0cno(d[n], t[n], 0.02))*year/dEpp
        # уменьшаем шаг если температура падает слишком резко
        if abs(dt/t[n]) > 1e-3 :
            dr = 0.7*dr
#         if abs(dt/t[n]) < 0.5e-3 :
#             dr = 1.1*dr
        if m[n]/M_sol > 1.2 : # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            raise Exception('Mass exceeded 1.2*M_sol')
        Tsur = pow(l[n]/sigma/S, 1/4)
        if t[n] < Tsur :
            break
        if p[n] < 0. : 
#             print ('Negative Pressure')
            break

    if (n == dim - 2) :
         raise Exception("Error, increase dim!")
    
  # Проверка поверхностной температуры:
    if debug :
        Tsur = pow(l[n]/4/pi/sigma/r[n]/r[n], 1/4)
        print('Tsur=%.0f T=%.0f' % (Tsur, t[n]))

    df = pd.DataFrame({'Mass': m[:n], 'Radius': r[:n], 'Temperature': t[:n], 'Density': d[:n], 'Luminosity': l[:n], 
                       'Hydrogen': x[:n], 'Opacity': k[:n], 'Convection': conv[:n], 'Pressure': p[:n], 'Velocity': v[:n]})

#     mas = np.zeros(dimM)
#     rad = np.zeros(dimM)
#     tem = np.zeros(dimM)
#     den = np.zeros(dimM)
#     lum = np.zeros(dimM)
#     hyd = np.zeros(dimM)
#     opa = np.zeros(dimM)
#     pre = np.zeros(dimM)
#     con = np.zeros(dimM)
#     iprev = -1
#     for j in range(n+1) :
#         i = int(m[j]/m[n]*dimM)
#         if i == dimM :
#             i = dimM - 1
#         if i > iprev :
#             iprev = i
#             count = 0
#             sd = 0.
#             sl = 0.
#             sh = 0.
#             so = 0.
#             sp = 0.
#             sc = 0.
#         mas[i] = m[n]*(i+1)/dimM
#         rad[i] = r[j]
#         tem[i] = t[j]
#         count += 1
#         sd += d[j]
#         sl += l[j]
#         sh += x[j]
#         so += k[j]
#         sp += p[j]
#         sc += conv[j]
#         den[i] = sd/count
#         lum[i] = sl/count
#         hyd[i] = sh/count
#         opa[i] = so/count
#         pre[i] = sp/count
#         con[i] = sc/count
#     den[0] = d[0]
#     rad[0] = r[0]
#     tem[0] = t[0]
#     pre[0] = p[0]
#     tem[-1] = t[n]
#     rad[-1] = r[n]
#     df = pd.DataFrame({'Mass':mas, 'Radius':rad, 'Temperature':tem, 'Density':den, 'Luminosity':lum, 'Hydrogen':hyd, 'Opacity':opa, 'Convection':con})
    return (n, m[n], r[n], l[n], t[n], df)

In [61]:
%%time
T0 = (13.9)*1e6
den0 = 90
n, M, R, L, T, df = model_R(den0, T0, debug=False)


Wall time: 799 ms


In [62]:
print('T0=%.5f D0=%.10f n=%d M=%.4f R=%.4f L=%.4f T=%.0f' % (T0, den0, n, M/M_sol, R/R_sol, L/L_sol, T))

T0=13900000.00000 D0=90.0000000000 n=10419 M=0.8090 R=0.4289 L=0.9254 T=8648


In [15]:
T0 = (13.31)*1e6
den0 = 95
n, M, R, L, T, df = model_R(den0, T0, debug=False)
n, M/M_sol, R/R_sol, L/L_sol, T

(10463,
 0.7710539902552994,
 0.4233843163046257,
 0.591054804683255,
 7780.355597636594)

### Поиск комбинации центральных плотности и температуры, дающих решение с массой равной солнечной. (Первый вариант)

Для интегрирования уравнений из центра надо задать плотность и температуру в центре, но только одна их комбинации 
соответствует решению с массой, равной солнечной (при фиксированном хим. составе)

Внешний цикл по температурам, внутренний цикл - по плотности.
При фиксированной температуре постепенно повышаем плотность, пока не появится стабильное решение с массой < 1.
После этого возвращаемся на 1 шаг назад по плотности, уменьшаем шаг по плотности в 10 раз и повторям процедуру.
Так делаем пока шаг по плотности не станет меньше 1e-8.

In [90]:
%%time
dim = 1 # 11
for j in range(dim) :
    # T0 = 1.333e7 + 0.001e7*j  # M = M_sol, epoch = 0
    T0 = 13.9e6 + 0.1e6*j
    step = 10 # step by density
    exc = True
    den0 = 30
    if j > 0:
        df_prev = df.copy()
    while step > 1e-8 :    
        for i in range(10) :
            try :
                n, M, R, L, T, df = model_R(den0, T0, debug=False)
            except Exception as error:
#                 print(T0, den0, 'Caught this error: ' + repr(error))
                exc = True
                den0 += step
                break
            if exc :
                exc = False
                print('\tT0=%.5e D0=%.10f n=%d M=%.4f R=%.4f L=%.4f T=%.0f' % (T0, den0, n, M/M_sol, R/R_sol, L/L_sol, T))
                den0 -= step
                step /= 10.
                break
            
    print('# T0=%.5e D0=%.2f n=%d M=%.4f R=%.4f L=%.3f T=%.0f' % (T0, den0, n, M/M_sol, R/R_sol, L/L_sol, T))
    if M/M_sol > 1 :
#         df = df_prev
        break
print("Done!")

	T0=1.39000e+07 D0=220.0000000000 n=9182 M=0.4972 R=0.2670 L=1.0253 T=11248
	T0=1.39000e+07 D0=212.0000000000 n=9410 M=0.5712 R=0.3006 L=1.1707 T=10960
	T0=1.39000e+07 D0=211.4000000000 n=10050 M=0.6669 R=0.4021 L=1.2177 T=9566
	T0=1.39000e+07 D0=211.4000000000 n=10050 M=0.6669 R=0.4021 L=1.2177 T=9566
	T0=1.39000e+07 D0=211.3950000000 n=10476 M=0.6919 R=0.4773 L=1.2185 T=8785
	T0=1.39000e+07 D0=211.3947000000 n=10948 M=0.7034 R=0.5562 L=1.2186 T=8135
	T0=1.39000e+07 D0=211.3946900000 n=11128 M=0.7056 R=0.5853 L=1.2186 T=7932
	T0=1.39000e+07 D0=211.3946870000 n=11301 M=0.7070 R=0.6107 L=1.2186 T=7762
	T0=1.39000e+07 D0=211.3946866000 n=11353 M=0.7073 R=0.6183 L=1.2186 T=7718
	T0=1.39000e+07 D0=211.3946865700 n=11358 M=0.7074 R=0.6190 L=1.2186 T=7709
# T0=1.39000e+07 D0=211.39 n=11358 M=0.7074 R=0.6190 L=1.219 T=7709
Done!
Wall time: 2min 8s


In [49]:
df = df_prev
df

,Mass,Radius,Temperature,Density,Luminosity,Hydrogen,Opacity,Convection,Pressure,Velocity
0,0.000000e+00,0.000000e+00,1.333000e+07,94.554471,0.000000e+00,0.732,1.318613,0.0,1.730205e+17,8.561869e-11
1,1.335360e+26,6.960000e+07,1.333000e+07,94.554471,1.634396e+27,0.732,1.318613,1.0,1.730205e+17,8.561869e-11
2,1.735967e+26,7.656000e+07,1.332997e+07,94.554040,2.124711e+27,0.732,1.318620,0.0,1.730193e+17,8.561741e-11
3,2.220700e+26,8.352000e+07,1.332993e+07,94.553578,2.717979e+27,0.732,1.318627,0.0,1.730180e+17,8.561603e-11
4,2.797568e+26,9.048000e+07,1.332990e+07,94.553080,3.424001e+27,0.732,1.318634,0.0,1.730166e+17,8.561455e-11
...,...,...,...,...,...,...,...,...,...,...
13995,1.989737e+33,6.262796e+10,5.465099e+03,0.000031,2.461129e+33,0.732,0.003113,0.0,2.349260e+07,4.899800e-49
13996,1.989737e+33,6.262800e+10,5.460450e+03,0.000031,2.461129e+33,0.732,0.003109,0.0,2.344258e+07,4.849587e-49
13997,1.989737e+33,6.262805e+10,5.455801e+03,0.000031,2.461129e+33,0.732,0.003105,0.0,2.339263e+07,4.799847e-49
13998,1.989737e+33,6.262810e+10,5.451153e+03,0.000031,2.461129e+33,0.732,0.003101,0.0,2.334274e+07,4.750576e-49


In [50]:
_ = df.to_csv(line_terminator='\n')
filename = "FromCenter/1epoch%.02d.csv" % epoch
print(filename)
csv_file = open(filename, "wt")
n = csv_file.write(_)
csv_file.close()

FromCenter/1epoch00.csv


### Поиск комбинации центральных плотности и температуры, дающих решение с массой равной солнечной. (Второй вариант)

Для интегрирования уравнений из центра надо задать плотность и температуру в центре, но только одна их комбинации соответствует решению с массой, равной солнечной (при фиксированном хим. составе)

Альтернативный поиск (в обратном порядке)

Внешний цикл по плотности, внутренний цикл - по температуре. При фиксированной плотности постепенно понижаем температуру, пока не появится стабильное решение с массой < 1. После этого возвращаемся на 1 шаг назад по температуре, уменьшаем шаг по температуре в 10 раз и повторям процедуру. Так делаем пока шаг по температуре не станет меньше 0.01 градуса.

In [ ]:
%%time
# Another order of fit - density outside
dim = 11
for j in range(dim) :
    den0 = 94 + 0.1*j # 94.4 + 0.1*j
    step = 1e6 # step for Temperature
    exc = True
    T0 = 15
    if j > 0:
        df_prev = df.copy()
    while step > 1e-2 :    
        for i in range(10) :
            try :
                n, M, R, L, T, df = model_R(den0, T0, debug=False)
            except Exception as error:
#                 print(T0, den0, 'Caught this error: ' + repr(error))
                exc = True
                T0 -= step
                break
            if exc :
                exc = False
                print('\tT0=%.5f D0=%.10f n=%d M=%.4f R=%.4f L=%.4f T=%.0f' % (T0, den0, n, M/M_sol, R/R_sol, L/L_sol, T))
                T0 += step
                step /= 10.
                break
            
    print('# T0=%.5f D0=%.2f n=%d M=%.4f R=%.4f L=%.3f T=%.0f' % (T0, den0, n, M/M_sol, R/R_sol, L/L_sol, T))
    if M/M_sol > 1. :
#         df = df_prev
        break
print("Done!")

In [ ]:
# df = df_prev
df

In [ ]:
_ = df.to_csv(line_terminator='\n')
filename = "FromCenter/2epoch%.02d.csv" % epoch
print(filename)
csv_file = open(filename, "wt")
n = csv_file.write(_)
csv_file.close()

In [30]:
T0=1.33000e+07 
den0=94.4000000000
n, M, R, L, T, df = model_R(den0, T0, debug=False)
n, M/M_sol, R/R_sol, L/L_sol, T

(11049,
 0.8772552888605576,
 0.5009900889800694,
 0.6252592215835585,
 7252.863270233664)